In [1]:
import pandas as pd
from src.executor.match_recognize import match_recognize
# Example with your original query
data = [
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", 
     "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob", "department": "Sales", "region": "West", 
     "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", 
     "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", 
     "hire_date": "2021-01-04", "salary": 1100},
    # Add more departments/regions
    {"id": 5, "name": "Eve", "department": "Sales", "region": "East", 
     "hire_date": "2021-01-01", "salary": 1250},
    {"id": 6, "name": "Frank", "department": "Marketing", "region": "West", 
     "hire_date": "2021-01-01", "salary": 1150},
]
df = pd.DataFrame(data)

query = """
SELECT id, name 
FROM employees 
MATCH_RECOGNIZE (
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES salary AS avg_salary
    PATTERN (A)
    DEFINE A AS salary > 1000
);
"""

result = match_recognize(query, df)
print("Match Recognize Output:")
print(result)



DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT id, name FROM employees MATCH_RECOGNIZE ( PARTITION BY department, region ORDER BY hire_date MEASURES salary AS avg_salary PATTERN (A) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=id, metadata={}), SelectItem(expression=name, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='employees')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='salary', alias='avg_salary', metada

Match Recognize Output:
   id   name department region   hire_date  salary  avg_salary  MATCH_NUMBER
0   6  Frank  Marketing   West  2021-01-01    1150        1150             1
1   5    Eve      Sales   East  2021-01-01    1250        1250             1
2   1  Alice      Sales   West  2021-01-01    1200        1200             1
3   2    Bob      Sales   West  2021-01-02    1300        1300             2
4   4  Diana      Sales   West  2021-01-04    1100        1100             3


In [4]:
import pandas as pd
from src.executor.match_recognize import match_recognize
data = {
    'order_id': [1, 2, 3, 4, 5, 6],
    'customer_name': ['John', 'John', 'Mary', 'John', 'Mary', 'Mary'],
    'order_date': pd.to_datetime(['01-JAN-2023', '02-JAN-2023', '02-JAN-2023',
                                   '03-JAN-2023', '04-JAN-2023', '05-JAN-2023']),
    'order_amount': [100, 200, 150, 120, 180, 80]
}

df = pd.DataFrame(data)


query = """
SELECT *
FROM   orders
MATCH_RECOGNIZE (
  PARTITION BY customer_name
  ORDER BY order_date
  MEASURES
    FIRST(order_date) AS start_date,
    LAST(order_date) AS end_date,
    SUM(order_amount) AS total_amount,
    COUNT(*) AS order_count
  PATTERN (A+)
  DEFINE A AS order_amount > 0
);
"""

result = match_recognize(query, df)
print("Match Recognize Output:")
print(result)



/tmp/ipykernel_7949/3405666025.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'order_date': pd.to_datetime(['01-JAN-2023', '02-JAN-2023', '02-JAN-2023',
DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE ( PARTITION BY customer_name ORDER BY order_date MEASURES FIRST(order_date) AS start_date, LAST(order_date) AS end_date, SUM(order_amount) AS total_amount, COUNT(*) AS order_count PATTERN (A+) DEFINE A AS order_amount > 0 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: Partition

ParserError: Referenced variable(s) {'order_date'} not found in the PATTERN clause or SUBSET definitions. (Line: 2, Column: 7)
Snippet: ordersMATCH_RECOGNIZE(PARTITIONBYcustomer_nameORDERBYorder_dateMEASURESFIRST(order_date)ASstart_date,LAST(order_date)ASend_date,SUM(order_amount)AStotal_amount,COUNT(*)ASorder_countPATTERN(A+)DEFINEAASorder_amount>0)

In [1]:
import pandas as pd
from src.executor.match_recognize import match_recognize
data = {
    'order_id': [1, 2, 3, 4, 5, 6],
    'customer': ['John', 'John', 'Mary', 'John', 'Mary', 'Mary'],
    'order_date': pd.to_datetime(['01-JAN-2023', '02-JAN-2023', '02-JAN-2023',
                                   '03-JAN-2023', '04-JAN-2023', '05-JAN-2023']),
    'amount': [100, 200, 150, 120, 180, 80]
}

df = pd.DataFrame(data)

# Corrected query with proper column names and pattern variable prefixes
query = """
SELECT *
FROM   orders
MATCH_RECOGNIZE (
  PARTITION BY customer
  ORDER BY order_date
  MEASURES
    FIRST(A.order_date) AS start_date,
    LAST(A.order_date) AS end_date,
    SUM(A.amount) AS total_amount,
    COUNT(*) AS order_count
  PATTERN (A+)
  DEFINE A AS amount > 0
);
"""

result = match_recognize(query, df)
print("Match Recognize Output:")
print(result)


/tmp/ipykernel_104695/4122375284.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'order_date': pd.to_datetime(['01-JAN-2023', '02-JAN-2023', '02-JAN-2023',
DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE ( PARTITION BY customer ORDER BY order_date MEASURES FIRST(A.order_date) AS start_date, LAST(A.order_date) AS end_date, SUM(A.amount) AS total_amount, COUNT(*) AS order_count PATTERN (A+) DEFINE A AS amount > 0 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(

Match Recognize Output:
   order_id customer order_date  amount start_date end_date  total_amount  \
0         1     John 2023-01-01     100       None     None           100   
1         2     John 2023-01-02     200       None     None           200   
2         4     John 2023-01-03     120       None     None           120   
3         3     Mary 2023-01-02     150       None     None           150   
4         5     Mary 2023-01-04     180       None     None           180   
5         6     Mary 2023-01-05      80       None     None            80   

   order_count  MATCH_NUMBER  
0            1             1  
1            1             2  
2            1             3  
3            1             1  
4            1             2  
5            1             3  


In [7]:
import pandas as pd
import numpy as np

# Create stock price data
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', periods=20, freq='D')
prices = [100]

# Generate a realistic price series with some patterns
for i in range(1, 20):
    if i == 5:  # Sharp drop
        prices.append(prices[-1] * 0.9)  # 10% drop
    elif i == 6:  # Slight recovery
        prices.append(prices[-1] * 1.03)  # 3% up
    elif i == 10:  # Peak
        prices.append(prices[-1] * 1.05)  # 5% up
    elif i == 11 or i == 12:  # Double down
        prices.append(prices[-1] * 0.97)  # 3% down
    elif i == 13 or i == 14:  # Double up
        prices.append(prices[-1] * 1.04)  # 4% up
    else:
        # Random movement
        prices.append(prices[-1] * (1 + np.random.normal(0, 0.01)))  # 1% volatility

# Calculate daily returns
returns = [0] + [(prices[i] - prices[i-1]) / prices[i-1] for i in range(1, len(prices))]

# Create the dataframe
stock_data = pd.DataFrame({
    'date': dates,
    'price': prices,
    'volume': np.random.randint(1000, 5000, size=len(prices)),
    'return': returns,
    'stock': ['AAPL'] * 10 + ['GOOG'] * 10  # Two different stocks
})

# Show the dataframe
print(stock_data)


         date       price  volume    return stock
0  2023-01-01  100.000000    1459  0.000000  AAPL
1  2023-01-02  100.496714    4385  0.004967  AAPL
2  2023-01-03  100.357763    1021 -0.001383  AAPL
3  2023-01-04  101.007769    3300  0.006477  AAPL
4  2023-01-05  102.546147    1747  0.015230  AAPL
5  2023-01-06   92.291533    3904 -0.100000  AAPL
6  2023-01-07   95.060279    4632  0.030000  AAPL
7  2023-01-08   94.837692    1474 -0.002342  AAPL
8  2023-01-09   94.615642    2082 -0.002341  AAPL
9  2023-01-10   96.109824    3558  0.015792  AAPL
10 2023-01-11  100.915315    4753  0.050000  GOOG
11 2023-01-12   97.887856    3047 -0.030000  GOOG
12 2023-01-13   94.951220    4547 -0.030000  GOOG
13 2023-01-14   98.749269    3747  0.040000  GOOG
14 2023-01-15  102.699240    1975  0.040000  GOOG
15 2023-01-16  103.487389    2806  0.007674  GOOG
16 2023-01-17  103.001542    1189 -0.004695  GOOG
17 2023-01-18  103.560388    4005  0.005426  GOOG
18 2023-01-19  103.080470    3734 -0.004634  GOOG


In [1]:
import pandas as pd
from src.executor.match_recognize import match_recognize
data = {
    'order_id': [1, 2, 3, 4, 5, 6],
    'customer': ['John', 'John', 'Mary', 'John', 'Mary', 'Mary'],
    'order_date': pd.to_datetime(['01-JAN-2023', '02-JAN-2023', '02-JAN-2023',
                                   '03-JAN-2023', '04-JAN-2023', '05-JAN-2023']),
    'amount': [100, 200, 150, 120, 180, 80]
}

df = pd.DataFrame(data)

# Modified query to test multi-row pattern matching
query = """
SELECT *
FROM   orders
MATCH_RECOGNIZE (
  PARTITION BY customer
  ORDER BY order_date
  MEASURES
    FIRST(A.order_date) AS start_date,
    LAST(A.order_date) AS end_date,
    SUM(A.amount) AS total_amount,
    COUNT(*) AS order_count
  ONE ROW PER MATCH
  PATTERN (A+)
  DEFINE A AS amount > 0
);
"""

result = match_recognize(query, df)
print("Match Recognize Output:")
print(result)


/tmp/ipykernel_112179/4131321436.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'order_date': pd.to_datetime(['01-JAN-2023', '02-JAN-2023', '02-JAN-2023',
DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM orders MATCH_RECOGNIZE ( PARTITION BY customer ORDER BY order_date MEASURES FIRST(A.order_date) AS start_date, LAST(A.order_date) AS end_date, SUM(A.amount) AS total_amount, COUNT(*) AS order_count ONE ROW PER MATCH PATTERN (A+) DEFINE A AS amount > 0 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='orders')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: 

Evaluating expression: FIRST(A.order_date)
Context variables: {'A': [0, 1, 2]}
Number of rows: 3
Evaluating navigation function: FIRST(A.order_date)
Function: FIRST, Args: ['A.order_date']
Evaluating expression: LAST(A.order_date)
Context variables: {'A': [0, 1, 2]}
Number of rows: 3
Evaluating navigation function: LAST(A.order_date)
Function: LAST, Args: ['A.order_date']
Evaluating expression: SUM(A.amount)
Context variables: {'A': [0, 1, 2]}
Number of rows: 3
Evaluating aggregate function: sum(A.amount)
Evaluating expression: COUNT(*)
Context variables: {'A': [0, 1, 2]}
Number of rows: 3
Evaluating aggregate function: count(*)
Evaluating expression: FIRST(A.order_date)
Context variables: {'A': [0, 1, 2]}
Number of rows: 3
Evaluating navigation function: FIRST(A.order_date)
Function: FIRST, Args: ['A.order_date']
Evaluating expression: LAST(A.order_date)
Context variables: {'A': [0, 1, 2]}
Number of rows: 3
Evaluating navigation function: LAST(A.order_date)
Function: LAST, Args: ['A.o

In [1]:
# test_match_recognize.py

import pandas as pd
from src.executor.match_recognize import match_recognize

def test_basic_pattern():
    """Test basic pattern matching with A+ pattern."""
    data = [
        {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
        {"id": 2, "name": "Bob", "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
        {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
        {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
    ]
    df = pd.DataFrame(data)
    
    query = """
    SELECT id, name 
    FROM employees 
    MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES salary AS avg_salary
        PATTERN (A)
        DEFINE A AS salary > 1000
    );
    """
    
    result = match_recognize(query, df)
    print("Basic Pattern Test:")
    print(result)

def test_price_trend():
    """Test price trend pattern (increasing then decreasing)."""
    data = [
        {"order_id": 1, "customer_id": "C1", "price": 100, "order_date": "2023-01-01"},
        {"order_id": 2, "customer_id": "C1", "price": 150, "order_date": "2023-01-02"},
        {"order_id": 3, "customer_id": "C1", "price": 200, "order_date": "2023-01-03"},
        {"order_id": 4, "customer_id": "C1", "price": 180, "order_date": "2023-01-04"},
        {"order_id": 5, "customer_id": "C1", "price": 160, "order_date": "2023-01-05"},
        {"order_id": 6, "customer_id": "C2", "price": 120, "order_date": "2023-01-01"},
        {"order_id": 7, "customer_id": "C2", "price": 140, "order_date": "2023-01-02"},
        {"order_id": 8, "customer_id": "C2", "price": 130, "order_date": "2023-01-03"},
    ]
    df = pd.DataFrame(data)
    
    query = """
    SELECT *
    FROM orders
    MATCH_RECOGNIZE (
        PARTITION BY customer_id
        ORDER BY order_date
        MEASURES 
            FIRST(A.price) AS start_price,
            LAST(A.price) AS peak_price,
            LAST(B.price) AS end_price
        ONE ROW PER MATCH
        AFTER MATCH SKIP TO NEXT ROW
        PATTERN (A+ B+)
        DEFINE
            A AS PREV(price) IS NULL OR price > PREV(price),
            B AS price < PREV(price)
    );
    """
    
    result = match_recognize(query, df)
    print("\nPrice Trend Test:")
    print(result)

def test_complex_pattern():
    """Test complex pattern with alternation and quantifiers."""
    data = [
        {"event_id": 1, "sequence": "A", "value": 10, "timestamp": "2023-01-01 10:00:00"},
        {"event_id": 2, "sequence": "B", "value": 20, "timestamp": "2023-01-01 10:01:00"},
        {"event_id": 3, "sequence": "A", "value": 30, "timestamp": "2023-01-01 10:02:00"},
        {"event_id": 4, "sequence": "C", "value": 40, "timestamp": "2023-01-01 10:03:00"},
        {"event_id": 5, "sequence": "B", "value": 50, "timestamp": "2023-01-01 10:04:00"},
        {"event_id": 6, "sequence": "A", "value": 60, "timestamp": "2023-01-01 10:05:00"},
    ]
    df = pd.DataFrame(data)
    
    query = """
    SELECT *
    FROM events
    MATCH_RECOGNIZE (
        ORDER BY timestamp
        MEASURES 
            FIRST(A.value) AS first_value,
            LAST(B.value) AS last_value,
            COUNT(*) AS pattern_length
        ALL ROWS PER MATCH
        PATTERN ((A B+) | (B A+))
        SUBSET U = (A, B)
        DEFINE
            A AS sequence = 'A',
            B AS sequence = 'B'
    );
    """
    
    result = match_recognize(query, df)
    print("\nComplex Pattern Test:")
    print(result)

def test_running_measures():
    """Test running measures and aggregations."""
    data = [
        {"id": 1, "metric": 100, "timestamp": "2023-01-01"},
        {"id": 2, "metric": 110, "timestamp": "2023-01-02"},
        {"id": 3, "metric": 120, "timestamp": "2023-01-03"},
        {"id": 4, "metric": 115, "timestamp": "2023-01-04"},
        {"id": 5, "metric": 125, "timestamp": "2023-01-05"},
    ]
    df = pd.DataFrame(data)
    
    query = """
    SELECT *
    FROM metrics
    MATCH_RECOGNIZE (
        ORDER BY timestamp
        MEASURES 
            RUNNING AVG(A.metric) AS running_avg,
            FINAL AVG(A.metric) AS final_avg,
            RUNNING COUNT(*) AS running_count
        ALL ROWS PER MATCH
        PATTERN (A+)
        DEFINE
            A AS PREV(metric) IS NULL OR metric >= PREV(metric)
    );
    """
    
    result = match_recognize(query, df)
    print("\nRunning Measures Test:")
    print(result)

def test_unmatched_rows():
    """Test handling of unmatched rows."""
    data = [
        {"id": 1, "value": 100, "category": "A"},
        {"id": 2, "value": 90, "category": "B"},
        {"id": 3, "value": 80, "category": "A"},
        {"id": 4, "value": 110, "category": "B"},
        {"id": 5, "value": 120, "category": "A"},
    ]
    df = pd.DataFrame(data)
    
    query = """
    SELECT *
    FROM data
    MATCH_RECOGNIZE (
        PARTITION BY category
        ORDER BY id
        MEASURES 
            FIRST(A.value) AS start_value,
            LAST(A.value) AS end_value
        ALL ROWS PER MATCH WITH UNMATCHED ROWS
        PATTERN (A+)
        DEFINE
            A AS PREV(value) IS NULL OR value > PREV(value)
    );
    """
    
    result = match_recognize(query, df)
    print("\nUnmatched Rows Test:")
    print(result)

if __name__ == "__main__":
    # Run all tests
    test_basic_pattern()
    test_price_trend()
    test_complex_pattern()
    test_running_measures()
    test_unmatched_rows()

DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT id, name FROM employees MATCH_RECOGNIZE ( PARTITION BY department, region ORDER BY hire_date MEASURES salary AS avg_salary PATTERN (A) DEFINE A AS salary > 1000 );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=id, metadata={}), SelectItem(expression=name, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='employees')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['department', 'region'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='hire_date', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='salary', alias='avg_salary', metada

AttributeError: 'dict' object has no attribute 'condition'